# Dense Overlap Score Demo
This notebook explains step by step how the dense overlap score is computed for a given pair of images.

### Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pycolmap
import matplotlib.pyplot as plt
from matplotlib.image import imread

from megadepth.utils.projections import backward_project, forward_project
from megadepth.utils.utils import camera_pixel_grid
from megadepth.utils.read_write_model import qvec2rotmat
from megadepth.utils.io import load_depth_map

In [ ]:
def plot_images(images: list, titles: list) -> None:
    fig = plt.figure(figsize=(30, 30))

    for i in range(len(images)):
        fig.add_subplot(1, len(images), i + 1)
        plt.axis("off")
        plt.imshow(images[i])
        plt.title(titles[i])

    plt.show()


def single_scatter_plot(x, y, c, title=""):
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    plt.gca().invert_yaxis()
    ax.scatter(x, y, c=c, s=1)
    plt.title(title)
    plt.show()


def multiple_scatter_plots(x: list, y: list, c: list, titles: list):
    n = len(x)
    fig, axes = plt.subplots(1, n, figsize=(8 * n, 6))
    for i in range(n):
        axes[i].invert_yaxis()
        axes[i].scatter(x[i], y[i], c=c[i], s=1)
        axes[i].title.set_text(titles[i])
    plt.show()

Feel free to change the variables in the following cell to try out different things:

In [ ]:
# path to the directory that contains the following subdirectories of the dense reconstruction:
# "image", "sparse", "stereo/depth_maps"
dense_model_dir = os.path.join("..", "..", "5018", "dense", "superpoint_max-superglue-netvlad-50")
sparse_model_dir = os.path.join("..", "..", "5018", "sparse", "superpoint_max-superglue-netvlad-50")

# image IDs
id_1 = 50
id_2 = 56

# by which factor to downsample the depth maps for the computation of the score
downsample = 10

# Dense points with an absolute relative depth error below this threshold are considered as inliers
rel_thresh = 0.03

In [ ]:
# load images, cameras and depth maps
image_dir = os.path.join(dense_model_dir, "images")
depth_map_dir = os.path.join(dense_model_dir, "stereo", "depth_maps")
normal_map_dir = os.path.join(dense_model_dir, "stereo", "normal_maps")

reconstruction = pycolmap.Reconstruction(os.path.join(dense_model_dir, "sparse"))

images = reconstruction.images
cameras = reconstruction.cameras
point3D = reconstruction.points3D

image_1 = images[id_1]
image_2 = images[id_2]
camera_1 = cameras[image_1.camera_id]
camera_2 = cameras[image_2.camera_id]
depth_map_1 = load_depth_map(os.path.join(depth_map_dir, f"{image_1.name}.geometric.bin"))
colors_1 = imread(os.path.join(image_dir, image_1.name))
depth_map_2 = load_depth_map(os.path.join(depth_map_dir, f"{image_2.name}.geometric.bin"))
colors_2 = imread(os.path.join(image_dir, image_2.name))

normal_map_1 = load_depth_map(os.path.join(normal_map_dir, f"{image_1.name}.geometric.bin"))
normal_map_2 = load_depth_map(os.path.join(normal_map_dir, f"{image_2.name}.geometric.bin"))

plot_images([colors_1, colors_2], ["Image 1", "Image 2"])
plot_images(
    [normal_map_1 * 0.5 + 0.5, depth_map_1, normal_map_2 * 0.5 + 0.5, depth_map_2],
    ["Normal Map 1", "Depth Map 1", "Normal Map 2", "Depth Map 2"],
)

plot_images([np.abs(normal_map_1[:, :, i]) for i in [0, 1, 2]], "xyz")
plot_images([np.abs(normal_map_2[:, :, i]) for i in [0, 1, 2]], "xyz")

### Downsampling
As a first step, we downsample the depth map. This speeds up the computation and is a fairly accurate estimation of the actual dense overlap score. We then filter out all invalid depth values.

In [ ]:
# gather depth values that we want to check in a vector
depth_1 = depth_map_1[::downsample, ::downsample].ravel()
normal_1 = normal_map_1[::downsample, ::downsample].reshape(-1, 3)

# get the corresponding 2D coordinates in image 1
points_2d = camera_pixel_grid(camera_1, downsample)


# filter out invalid depth values
valid_depth_mask = depth_1 > 0.0
depth_1 = depth_1[valid_depth_mask]
normal_1 = normal_1[valid_depth_mask]
points_2d = points_2d[valid_depth_mask]

# number of dense features we are considering for the score computation
n_features = depth_1.size

print(f"{n_features} dense features in image 1 will be considered for the overlap score")

In [ ]:
print("We consider the following subsampled pixels for the computation of the dense overlap score:")

# Note: the color values are stored column-wise and not row-wise!
rgb = colors_1[points_2d[:, 1].astype(int), points_2d[:, 0].astype(int)] / 255
multiple_scatter_plots(
    x=2 * [points_2d[:, 0]],
    y=2 * [points_2d[:, 1]],
    c=[depth_1, normal_1 * 0.5 + 0.5, rgb],
    titles=["Depth values", "Normal values", "RGB values"],
)

### Backprojection
Next, we backproject all dense features from image 1 to 3D using the corresponding (valid) depth values in the first depth map.

In [ ]:
# backproject all valid 2D points from image 1 to 3D
points_3d = backward_project(
    points_2d=points_2d,
    image=image_1,
    camera=camera_1,
    depth=depth_1,
)

print("Dense features from image 1 in 3D:")
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw={"projection": "3d"})
ax.scatter(points_3d[:, 0], points_3d[:, 1], points_3d[:, 2], s=1, c=rgb)
plt.show()

### Forward Projection
Then, we project all backprojected 3D points from image 1 to the image space of image 2. All invalid points, i.e. points that lie outside of image 2, are discarded. We calculate the absolute relative depth error between the depth values obtained through the 3D-2D projection and the corresponding depth values in the second depth map.

In [ ]:
# project all 3D points to image 2 to obtain 2D points and associated depth values
proj_points_2d, proj_depths, proj_mask = forward_project(
    points_3d=points_3d, image=image_2, camera=camera_2
)

# get corresponding depth values from the second depth map
depth_2 = depth_map_2[proj_points_2d[:, 1], proj_points_2d[:, 0]]

# compute absolute relative depth errors
abs_rel_error = np.abs(depth_2 / proj_depths - 1.0)

print("These are the dense features from image 1 that are successfully projected to image 2:")

multiple_scatter_plots(
    x=3 * [proj_points_2d[:, 0]],
    y=3 * [proj_points_2d[:, 1]],
    c=[proj_depths, depth_2, abs_rel_error],
    titles=[
        "Depth values obtained from 3D-2D projection",
        "Depth values from the second depth map",
        "Absolute relative depth error",
    ],
)


# RGB values from image 1 and image 2
# Note: the depth map values are stored column-wise and not row-wise!
rgb_1 = (
    colors_1[points_2d[proj_mask][:, 1].astype(int), points_2d[proj_mask][:, 0].astype(int)] / 255
)
rgb_2 = colors_2[proj_points_2d[:, 1].astype(int), proj_points_2d[:, 0].astype(int)] / 255

multiple_scatter_plots(
    x=2 * [points_2d[proj_mask][:, 0]],
    y=2 * [points_2d[proj_mask][:, 1]],
    c=[rgb_1, rgb_2],
    titles=[
        "RGB values from image 1",
        "RGB values from image 2",
    ],
)

Here is a histogram that shows the absolute relative depth errors between the asscociated depth values obtained through the projections from 3D to 2D and the actual depth values stored in the second depth map.

If a dense feature from image 1 lands on a pixel in image 2 that has an invalid depth value 0.0, the absolute error is 100% because: $$ abs({depth \over x} - 1.0) = 1.0 $$

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.hist(abs_rel_error, bins=100)
plt.show()

### Thresholding
Finally, we threshold the dense features that were successfully projected to image 2 based on the absolute relative depth error to obtain the final dense overlap score.

In [ ]:
n_inliners = np.count_nonzero(abs_rel_error < rel_thresh)
score = n_inliners / n_features

print(f"{n_features} dense features in image 1 were considered for the overlap score")
print(
    f"{proj_points_2d.shape[0]} features were succesfully projected to image 2 (i.e. they lie inside image 2)"
)
print(
    f"{n_inliners} features have an acceptable absolute relative depth error (less than {rel_thresh * 100}%)"
)

print(f"\n==> dense overlap score: {score}")

cos_w = np.clip(-normal_map_2[proj_points_2d[:, 1], proj_points_2d[:, 0], 2], 0, 1)
print(np.sum(cos_w[abs_rel_error < rel_thresh]) / n_features)

In [ ]:
from megadepth.metrics.overlap import dense_overlap

dense_overlap_matrix = dense_overlap(reconstruction, depth_map_dir, 50)

In [ ]:
from megadepth.metrics.overlap import dense_overlap

dense_overlap_matrix_cosine_weighted = dense_overlap(
    reconstruction, depth_map_dir, 50, cosine_weighted=True, normal_path=normal_map_dir
)

In [ ]:
from megadepth.metrics.overlap import sparse_overlap

sparse_overlap_matrix = sparse_overlap(reconstruction)

In [ ]:
from megadepth.metrics.angle import angle

angle_matrix = angle(reconstruction)
fig = plt.figure(figsize=(8, 6))
plt.scatter(angle_matrix.ravel(), sparse_overlap_matrix.ravel(), alpha=0.1)
plt.scatter(angle_matrix.ravel(), dense_overlap_matrix.ravel(), alpha=0.1)
plt.scatter(angle_matrix.ravel(), dense_overlap_matrix_cosine_weighted.ravel(), alpha=0.1)
plt.xlabel("$cos \Theta $")
plt.ylabel("Score")
plt.legend(["sparse overlap", "dense overlap", "cosine weighted dense overlap"])
plt.tight_layout()
plt.savefig(
    "scatter.png",
)

In [ ]:
plt.hist(dense_overlap_matrix.ravel(), alpha=0.5)
plt.hist(dense_overlap_matrix_cosine_weighted.ravel(), alpha=0.5)
plt.legend(["dense", "dense cos-w"])
plt.show()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

fig = plt.figure()
color_map = LinearSegmentedColormap.from_list("custom_map", ["white", "tab:orange"])

plt.tight_layout()
plt.axis("off")
plt.subplot(1, 4, 1)
plt.title("sparse overlap")
plt.tight_layout()
plt.axis("off")
plt.imshow(sparse_overlap_matrix, interpolation="nearest", cmap=color_map)
plt.subplot(1, 4, 2)
plt.title("dense overlap")
plt.tight_layout()
plt.axis("off")
plt.imshow(dense_overlap_matrix, interpolation="nearest", cmap=color_map)
plt.subplot(1, 4, 3)
plt.title("dense overlap cos w")
plt.tight_layout()
plt.axis("off")
plt.imshow(dense_overlap_matrix_cosine_weighted, interpolation="nearest", cmap=color_map)
plt.subplot(1, 4, 4)
plt.title("diff")
plt.tight_layout()
plt.axis("off")
plt.imshow(
    dense_overlap_matrix_cosine_weighted - dense_overlap_matrix,
    interpolation="nearest",
    cmap=color_map,
)
plt.show()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

color_map = LinearSegmentedColormap.from_list("custom_map", ["white", "tab:orange"])
matrices = [sparse_overlap_matrix, dense_overlap_matrix, dense_overlap_matrix_cosine_weighted]
names = ["sparse_mat.png", "cosw_mat.png", "dense_mat.png"]
for matrix, path in zip(matrices, names):
    fig = plt.figure(figsize=(10, 10))
    plt.tight_layout()
    plt.axis("off")
    plt.imshow(matrix * 0.2, interpolation="nearest", cmap=color_map)
    plt.show()
    # fig.savefig(path, dpi=600, bbox_inches="tight")
    plt.close(fig)

In [ ]:
plt.xlabel("sparse")
plt.ylabel("dense")
plt.scatter(sparse_overlap_matrix.ravel(), dense_overlap_matrix.ravel())
plt.show()

In [ ]:
discrepancy = np.abs(sparse_overlap_matrix - dense_overlap_matrix)

i, j = np.indices(discrepancy.shape)
i = i.ravel()
j = j.ravel()
d = discrepancy.ravel()
# this ranks the images first, where the two scores are closest, ignoring the cases where both are very small or very big
rank = np.argsort(-d + (1 - (1 - dense_overlap_matrix.ravel()) * dense_overlap_matrix.ravel()))

d = d[rank]
i = i[rank]
j = j[rank]
keys = list(images.keys())
for f in range(3):
    print(
        f"discrepancy: {discrepancy[i[f],j[f]]}, sparse: {sparse_overlap_matrix[i[f],j[f]]}, dense: {dense_overlap_matrix[i[f],j[f]]}"
    )
    image_1 = images[keys[i[f]]]
    image_2 = images[keys[j[f]]]
    camera_1 = cameras[image_1.camera_id]
    camera_2 = cameras[image_2.camera_id]
    colors_1 = imread(os.path.join(image_dir, image_1.name))
    colors_2 = imread(os.path.join(image_dir, image_2.name))
    plot_images([colors_1, colors_2], ["i", "j"])

In [ ]:
discrepancy = np.abs(dense_overlap_matrix_cosine_weighted - dense_overlap_matrix)
for i in range(discrepancy.shape[0]):
    discrepancy[i, i] = 0

i, j = np.indices(discrepancy.shape)
i = i.ravel()
j = j.ravel()
d = discrepancy.ravel()
# this ranks the images first, where the two scores differ the most
rank = np.argsort(
    -d
    + (
        1
        - (1 - dense_overlap_matrix_cosine_weighted.ravel())
        * dense_overlap_matrix_cosine_weighted.ravel()
    )
)
d = d[rank]
i = i[rank]
j = j[rank]
keys = list(images.keys())
for f in range(3):
    print(
        f"discrepancy: {discrepancy[i[f],j[f]]}, sparse: {sparse_overlap_matrix[i[f],j[f]]}, dense_cos_w: {dense_overlap_matrix_cosine_weighted[i[f],j[f]]}, dense: {dense_overlap_matrix[i[f],j[f]]}"
    )
    print(
        f"discrepancy: {discrepancy[j[f],i[f]]}, sparse: {sparse_overlap_matrix[i[f],j[f]]}, dense_cos_w: {dense_overlap_matrix_cosine_weighted[j[f],i[f]]}, dense: {dense_overlap_matrix[j[f],i[f]]}"
    )
    image_1 = images[keys[i[f]]]
    image_2 = images[keys[j[f]]]
    print(keys[i[f]], keys[j[f]])
    camera_1 = cameras[image_1.camera_id]
    camera_2 = cameras[image_2.camera_id]
    colors_1 = imread(os.path.join(image_dir, image_1.name))
    colors_2 = imread(os.path.join(image_dir, image_2.name))
    plot_images([colors_1, colors_2], ["i", "j"])

In [ ]:
from megadepth.visualization.view_overlap import vis_overlap
from megadepth.visualization.view_projections import pca, create_view_projection_figure
from megadepth.utils.projections import get_camera_poses

camera_poses = get_camera_poses(reconstruction)
points = np.array([p.xyz for p in reconstruction.points3D.values()])
align = pca(camera_poses)
# create_view_projection_figure( [align(points),align(camera_poses)], limit=3,alpha=.5)
# create_view_projection_figure([align(points),align(camera_poses)],view=0, limit=2,alpha=.5)

In [ ]:
print("cosine weighted dense overlap")
vis_overlap(
    dense_overlap_matrix_cosine_weighted,
    align(camera_poses),
    align(points),
    path="cos_w2.png",
    opacity=0.3,
)
print("dense overlap")
vis_overlap(
    dense_overlap_matrix, align(camera_poses), align(points), path="dense2.png", opacity=0.3
)
print("sparse overlap")
vis_overlap(
    sparse_overlap_matrix, align(camera_poses), align(points), path="sparse2.png", opacity=0.3
)

In [ ]:
print("sparse overlap")
vis_overlap(angle_matrix, align(camera_poses), align(points))
vis_overlap(angle_matrix, align(camera_poses), align(points))
i_matrix = np.zeros_like(dense_overlap_matrix)
i_matrix[:, [0, 50, 70]] = dense_overlap_matrix[:, [0, 50, 70]]
vis_overlap(i_matrix, align(camera_poses), align(points), opacity=1)
print("sparse overlap")
vis_overlap(sparse_overlap_matrix, align(camera_poses), align(points))
print("dense overlap")
vis_overlap(dense_overlap_matrix, align(camera_poses), align(points))
print("cosine weighted dense overlap")
vis_overlap(dense_overlap_matrix_cosine_weighted, align(camera_poses), align(points))
print("difference of cosine weight")
vis_overlap(
    dense_overlap_matrix_cosine_weighted - angle_matrix * dense_overlap_matrix,
    align(camera_poses),
    align(points),
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import cv2


def getImage(path, zoom=0.015):
    img = cv2.imread(path)[:, :, ::-1]
    # img = cv2.resize(img,(100,100))
    return OffsetImage(img, zoom=zoom)


fig, ax = plt.subplots()


matrix = dense_overlap_matrix

n = matrix.shape[0]
k = 5
i = 0
j = np.array(range(n))
dense = matrix[i, j]
dense_w = matrix[i, j]
sparse = matrix[i, j]
keys = list(images.keys())
img_list = []
for jj in j:
    image = images[keys[jj]]
    colors_1 = os.path.join(image_dir, image.name)
    img_list += [colors_1]

cam = align(camera_poses)
x_y = cam[j, :2]
x = matrix[j, i]
y = matrix[i, j]
print(x_y.shape)

fig = plt.figure(figsize=(10, 10), dpi=2000)
ax.set_xlabel("Dense( : , R )")  # Set x-axis label
ax.set_ylabel("Dense( R , : )")  # Set y-axis label

ax.scatter(x, y)
for x0, y0, path in zip(x, y, img_list):
    ab = AnnotationBbox(getImage(path), (x0, y0), frameon=False)
    ax.add_artist(ab)

# fig.savefig("overlap.png", dpi=600, bbox_inches="tight")
plt.show()


"""
#plot_images_on_scatter(img_list, dense,dense_w, 100)

plt.scatter(dense_overlap_matrix.ravel(),sparse_overlap_matrix.ravel(), alpha = 0.01)
plt.scatter(dense_overlap_matrix.ravel(),dense_overlap_matrix_cosine_weighted.ravel(), alpha = 0.01)
plt.scatter(dense_overlap_matrix_cosine_weighted.ravel(),sparse_overlap_matrix.ravel(), alpha = 0.01)

plt.scatter(dense.ravel(),sparse.ravel(),color='tab:blue')
plt.scatter(dense.ravel(),dense_w.ravel(), color = 'tab:orange')
plt.scatter(dense_w.ravel(),sparse.ravel(),color='tab:green')
x = np.stack([dense,dense,dense_w])
y = np.stack([dense_w,sparse,sparse])
plt.plot(x,y,color="black", linewidth=1)
plt.show()"""

In [ ]:
np.save("cosw.npy", dense_overlap_matrix_cosine_weighted)
np.save("dense.npy", dense_overlap_matrix)

In [ ]:
dense_overlap_matrix_cosine_weighted = np.load("cosw.npy")
dense_overlap_matrix = np.load("dense.npy")